In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
raw_data_path=os.path.join('data','raw')
df=pd.read_csv(os.path.join(raw_data_path,'fer2013.csv'))
X=[]
for i in range(len(df)):
    X.append(df.loc[i,'pixels'].split(' '))
X=np.array(X,np.float32)/255.
y=df.emotion.values
X_train=X[np.where(df.Usage=='Training')]
y_train=y[np.where((df.Usage=='Training'))]
X_test=X[np.where((df.Usage!='Training'))]
y_test=y[np.where((df.Usage!='Training'))]
def balance_classes(X,y):
    X1=X[np.where(y!=1)]
    X2=X[np.where(y==1)]
    X2=np.repeat(X2,9,axis=0)
    X=np.concatenate((X1,X2),axis=0)
    y1=y[np.where(y!=1)]
    y2=y[np.where(y==1)]
    y2=np.repeat(y2,9,axis=0)
    y=np.concatenate((y1,y2),axis=0)
    i=[i for i in range(len(y))]
    np.random.shuffle(i)
    return X[i],y[i]
X_train,y_train=balance_classes(X_train,y_train)
def one_hot_encoder(X):
    N=len(X)
    K=len(set(X))
    Z=np.zeros((N,K))
    for i in range(N):
        c=X[i]
        Z[i,c]=1
    return Z
y_train=one_hot_encoder(y_train)

In [2]:
import tensorflow as tf

C:\Users\ytnvj\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
class TF_ANN(object):
    def __init__(self,lr,epoch,hu):
        self.hu=hu
        self.lr=lr
        self.epoch=epoch
        pass
    def init_weights(self,shape):
        return tf.Variable(tf.random_normal(shape,stddev=0.01))
    def forward(self,X,W1,b1,W2,b2):
        z=tf.nn.sigmoid(tf.matmul(X,W1)+b1)
        return (tf.matmul(z,W2)+b2)
    def fit(self,X,y,fig=True):
        N,D=X.shape
        K=y.shape[1]
        W1=self.init_weights([D,self.hu])
        b1=self.init_weights([self.hu])
        W2=self.init_weights([self.hu,K])
        b2=self.init_weights([K])
        tr_i=np.random.choice(N,int(N*0.9),replace=False)
        t_i=[i for i in range(N) if i not in tr_i]
        X_train=X[tr_i]
        y_train=y[tr_i]
        X_test=X[t_i]
        y_test=y[t_i]
        c_t=[]
        c_v=[]
        cl_t=[]
        cl_v=[]
        tfX=tf.placeholder(tf.float32,[None,D])
        tfY=tf.placeholder(tf.float32,[None,K])
        y_p=self.forward(tfX,W1,b1,W2,b2)
        cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tfY,logits=y_p))
        optimizer=tf.train.GradientDescentOptimizer(self.lr)
        train=optimizer.minimize(cost)
        with tf.Session() as s:
            s.run(tf.global_variables_initializer())
            s.run(y_p,feed_dict={tfX:X_train})
            for i in range(self.epoch):
                c_t.append(s.run(cost,feed_dict={tfX:X_train,tfY:y_train}))
                c_v.append(s.run(cost,feed_dict={tfX:X_test,tfY:y_test})) 
                y=s.run(tf.argmax(tfY,1),feed_dict={tfY:y_train})
                yp=s.run(tf.argmax(y_p,1),feed_dict={tfX:X_train})
                cl_t.append(self.score(y,yp))  
                y=s.run(tf.argmax(tfY,1),feed_dict={tfY:y_test})
                yp=s.run(tf.argmax(y_p,1),feed_dict={tfX:X_test})
                cl_v.append(self.score(y,yp))
                s.run(train,feed_dict={tfX:X_train,tfY:y_train})
                if i % 100==0:
                    print('Epoch {0} Train Cost: {1} , Classification: {2} , Test Cost {3} , Classification : {4}'.format(i,c_t[i],
                                                                                                                      cl_t[i],
                                                                                                                      c_v[i],
                                                                                                                      cl_v[i]))
            self.W1=s.run(W1)
            self.W2=s.run(W2)
            self.b1=s.run(b1)
            self.b2=s.run(b2)
        if fig == True:
            plt.plot(c_t,label='Train Cost')
            plt.plot(c_v,label='Test Cost')
            plt.show()
            plt.plot(cl_t,label='Train Class')
            plt.plot(cl_v,label='Test Class')
            plt.show()
    def predict(self,X):
        act = self.forward(X,self.W1,self.b1,self.W2,self.b2)
        return tf.argmax(act, 1)
    def score(self,y,y_p):
        return np.mean(y==y_p)

In [9]:
ann=TF_ANN(0.0001,10000,5)

In [10]:
ann.fit(X_train,y_train)

Epoch 0 Train Cost: 1.9450747966766357 , Classification: 0.12585843945197916 , Test Cost 1.9448368549346924 , Classification : 0.13167701863354037
Epoch 100 Train Cost: 1.944872498512268 , Classification: 0.12799806743279152 , Test Cost 1.9446278810501099 , Classification : 0.12888198757763975
Epoch 200 Train Cost: 1.9446709156036377 , Classification: 0.12879180039341548 , Test Cost 1.9444199800491333 , Classification : 0.13105590062111802
Epoch 300 Train Cost: 1.9444705247879028 , Classification: 0.1344514615039514 , Test Cost 1.9442133903503418 , Classification : 0.13167701863354037
Epoch 400 Train Cost: 1.9442715644836426 , Classification: 0.1396624909410912 , Test Cost 1.9440077543258667 , Classification : 0.1357142857142857
Epoch 500 Train Cost: 1.94407320022583 , Classification: 0.14521862166545882 , Test Cost 1.9438031911849976 , Classification : 0.13944099378881988
Epoch 600 Train Cost: 1.9438762664794922 , Classification: 0.14808296234944957 , Test Cost 1.943599820137024 , Cla

KeyboardInterrupt: 

In [18]:
y_train.shape

(32197, 7)